In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from forecast_combine.reconcile import ForecastReconciler


In [2]:
# trained models for each autonomous factor
output_folder = '../model_output/'
model_names = ['CIC', 'NFA', 'SAB', 'Total_AF']
forecasters_d = {}
for model_name in model_names:
    with open(output_folder + model_name + '_forecast.pkl', 'rb') as f:
        forecasters_d[model_name] = pickle.load(f)    

In [3]:
# Hierarchical relationship matrix
S = pd.DataFrame(np.array([[1,1,-1], [1,0,0], [0,1,0], [0,0,1]]), 
                 columns=['CIC', 'SAB', 'NFA'], 
                 index=['Total_AF', 'CIC', 'SAB', 'NFA'])

In [4]:
# Hiearchical reconciliation model
reconciler = ForecastReconciler(forecasters_d=forecasters_d, S = S)

In [5]:
for m in ['ols', 'wls', 'mint', 'bu', 'td']:
    reconciled_preds, reconciled_intervals = reconciler.predict(reconciliation_method = m)
    reconciler.plot_predict(reconciled_preds, reconciled_intervals, title = m)
    plt.show()

LOESS does not support prediction intervals
Error: StatsForecastMSTL does not have the capability to return prediction intervals. If you think this estimator should have the capability, please open an issue on sktime.
